In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Load Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Sample dataset with both sentiment and intent labels
texts = [
    "I have heard good things about this from my friend. I want to buy it!",  # (Positive, Purchase Intent)
    "This is the worst thing I've ever bought.",  # (Negative, Complaint)
    "Excellent quality, highly recommended you to buy it",  # (Positive, Purchase Intent)
    "I am not satisfied at all, it has been terrible experience.",  # (Negative, Complaint)
    "I am very happy with this product in past would like to purchase again!",  # (Positive, Purchase Intent)
    "I hate this product, completely useless.",  # (Negative, Complaint)
    "Great value for money!",  # (Positive, Inquiry)
    "I am Disappointed, would not buy this product again."  # (Negative, Inquiry)
]

## **TASK 1: Sentence Transformer Implementation**

In [ ]:
#TASK
embeddings = np.array(model.encode(texts))

len(embeddings[0])

384

In [ ]:
len(embeddings)

8

Each sentence become embedding becomes length of size 384

# **TASK 2 Multi Task learning Expansion**

In [ ]:
# Labels for two tasks
sentiment_labels = [1, 0, 1, 0, 1, 0, 1, 0]  # 1 = Positive, 0 = Negative
intent_labels = ["purchase", "complaint", "purchase", "complaint", "purchase", "complaint", "inquiry", "inquiry"]

# Convert text to embeddings
embeddings = np.array(model.encode(texts))

# Encode intent labels as integers
intent_encoder = LabelEncoder()
intent_labels_encoded = intent_encoder.fit_transform(intent_labels)  # Converts to 0, 1, 2 classes

# Split data into training and testing sets
X_train, X_test, y_train_sentiment, y_test_sentiment, y_train_intent, y_test_intent = train_test_split(
    embeddings, sentiment_labels, intent_labels_encoded, test_size=0.2, random_state=42
)

# Define input layer
input_layer = keras.layers.Input(shape=(embeddings.shape[1],))

# Shared hidden layers
hidden = keras.layers.Dense(128, activation="relu")(input_layer)
hidden = keras.layers.Dropout(0.3)(hidden)
hidden = keras.layers.Dense(64, activation="relu")(hidden)

# Sentiment classification head
sentiment_head = keras.layers.Dense(1, activation="sigmoid", name="sentiment_output")(hidden)

# Intent classification head
intent_head = keras.layers.Dense(len(set(intent_labels_encoded)), activation="softmax", name="intent_output")(hidden)

# Define multi-output model
multi_output_model = keras.Model(inputs=input_layer, outputs=[sentiment_head, intent_head])

# Compile the model with separate loss functions
multi_output_model.compile(
    optimizer="adam",
    loss={
        "sentiment_output": "binary_crossentropy",
        "intent_output": "sparse_categorical_crossentropy",
    },
    metrics={
        "sentiment_output": "accuracy",
        "intent_output": "accuracy",
    },
)




To support multi-task learning, the architecture was modified to have a shared hidden layers followed by two separate output heads—one for sentiment classification and another for intent classification.

The Sentence Transformer (all-MiniLM-L6-v2) generates vector embeddings from input text.
A shared feedforward neural network processes these embeddings.
This shared hidden representation allows both tasks to leverage common linguistic features, improving efficiency and generalization.





## TASK 3  

```


1.   If the entire network is frozen network does not learn anything only inference is possible so this may not be viable option
2.   When only transformer architecture is frozen we depend on transformer to generate the fixed embeddings which are then passed to 2 heads and only task specific heads are trained.
3.   If one of the task-specific heads is frozen, that particular task does not learn anything.

In our case we have frozen the tranformer layers and we are training the task specific heads


Choice of Pre-Trained Model

1.   Choice of Pre-Trained Model

General NLP: all-MiniLM-L6-v2 (efficient, strong embeddings).
Domain-Specific: SciBERT for science , BioBERT for Biology, BloombergGPT for finance
2.   Layers to Freeze/Unfreeze

Step 1: Freeze Transformer Backbone (preserves pre-trained knowledge, ideal for small data).
Step 2: Train Task-Specific Heads (adapts to new tasks).
Step 3: Unfreeze Top Transformer Layers (if more adaptation is needed).

3.   Recommendations

Small dataset → Freeze backbone, train heads.
Large dataset → Gradually unfreeze top layers.



```



## TASK 4 - Training

In [ ]:
# Train the model
multi_output_model.fit(
    X_train, {"sentiment_output": np.array(y_train_sentiment), "intent_output": np.array(y_train_intent)},
    epochs=10, batch_size=4, validation_data=(X_test, {"sentiment_output": np.array(y_test_sentiment), "intent_output": np.array(y_test_intent)})
)

# Evaluate on test set
test_results = multi_output_model.evaluate(X_test, {"sentiment_output": np.array(y_test_sentiment), "intent_output": np.array(y_test_intent)})
print(f"Test Sentiment Accuracy: {test_results[3]:.2f}, Test Intent Accuracy: {test_results[4]:.2f}")

# Predict on new sample
new_text = ["I'm looking for a great laptop deal."]
new_embedding = np.array(sbert_model.encode(new_text))

sentiment_pred, intent_pred = multi_output_model.predict(new_embedding)
sentiment_label = "Positive" if sentiment_pred[0][0] > 0.5 else "Negative"
intent_label = intent_encoder.inverse_transform([np.argmax(intent_pred[0])])[0]

print(f"Predicted Sentiment: {sentiment_label}, Predicted Intent: {intent_label}")

Epoch 1/10
2/2 [==============================] - 0s 141ms/step - loss: 1.3571 - sentiment_output_loss: 0.5371 - intent_output_loss: 0.8200 - sentiment_output_accuracy: 0.6667 - intent_output_accuracy: 0.8333 - val_loss: 2.0306 - val_sentiment_output_loss: 0.7553 - val_intent_output_loss: 1.2753 - val_sentiment_output_accuracy: 0.0000e+00 - val_intent_output_accuracy: 0.0000e+00
Epoch 2/10
2/2 [==============================] - 0s 75ms/step - loss: 1.2918 - sentiment_output_loss: 0.4919 - intent_output_loss: 0.7999 - sentiment_output_accuracy: 1.0000 - intent_output_accuracy: 0.6667 - val_loss: 2.0577 - val_sentiment_output_loss: 0.7599 - val_intent_output_loss: 1.2978 - val_sentiment_output_accuracy: 0.0000e+00 - val_intent_output_accuracy: 0.0000e+00
Epoch 3/10
2/2 [==============================] - 0s 65ms/step - loss: 1.1799 - sentiment_output_loss: 0.4563 - intent_output_loss: 0.7236 - sentiment_output_accuracy: 1.0000 - intent_output_accuracy: 0.8333 - val_loss: 2.0879 - val_sent